In [1]:
complete_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest.zip'
small_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'

In [4]:
import os

datasets_path = os.path.join('..', 'datasets')

complete_dataset_path = os.path.join(datasets_path, 'ml-latest.zip')
small_dataset_path = os.path.join(datasets_path, 'ml-latest-small.zip')

In [ ]:
import urllib

small_f = urllib.urlretrieve (small_dataset_url, small_dataset_path)
complete_f = urllib.urlretrieve (complete_dataset_url, complete_dataset_path)

In [ ]:

import zipfile

with zipfile.ZipFile(small_dataset_path, "r") as z:
    z.extractall(datasets_path)

with zipfile.ZipFile(complete_dataset_path, "r") as z:
    z.extractall(datasets_path)

In [3]:
# import os
# import sys
# import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = "--master local[1] pyspark-shell"
# os.environ["SPARK_HOME"] = "/Users/osuba/Documents/spark-2.0.2-bin-hadoop2.7"
# os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.3-src.zip")
# sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

In [2]:
#Initializing PySpark
# from pyspark import SparkContext, SparkConf

# # #Spark Config
# conf = SparkConf().setAppName("recoomendation systems")
sc = SparkContext()


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PySparkShell, master=local[*]) created by <module> at //anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/IPython/utils/py3compat.py:289 

In [5]:
small_ratings_file = os.path.join(datasets_path, 'ml-latest-small', 'ratings.csv')

small_ratings_raw_data = sc.textFile(small_ratings_file)
small_ratings_raw_data_header = small_ratings_raw_data.first() #extract header
small_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header).map(lambda line: line.split(",")).map(lambda tokens:(tokens[0],tokens[1],tokens[2])).cache()


In [6]:
print small_ratings_raw_data_header
small_ratings_data.take(3)

userId,movieId,rating,timestamp


[(u'1', u'31', u'2.5'), (u'1', u'1029', u'3.0'), (u'1', u'1061', u'3.0')]

In [7]:
small_movies_file=os.path.join(datasets_path, 'ml-latest-small','movies.csv')
small_movies_raw_data=sc.textFile(small_movies_file)
small_movies_raw_data_header=small_movies_raw_data.first()
small_movies_data=small_movies_raw_data.filter(lambda line: line!=small_movies_raw_data_header).map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1])).cache()

In [8]:
small_ratings_raw_data_header

u'userId,movieId,rating,timestamp'

In [9]:
training_RDD, validation_RDD, test_RDD = small_ratings_data.randomSplit([6, 2, 2], seed=0L)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [12]:
training_RDD.take(3)


[(u'1', u'1029', u'3.0'), (u'1', u'1061', u'3.0'), (u'1', u'1263', u'2.0')]

In [11]:
validation_for_predict_RDD.take(3)

[(u'1', u'1129'), (u'1', u'1172'), (u'1', u'1405')]

In [13]:
from pyspark.mllib.recommendation import ALS
import math

seed = 5L
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print 'For rank %s the RMSE is %s' % (rank, error)
    if error < min_error:
        min_error = error
        best_rank = rank

print 'The best model was trained with rank %s' % best_rank

For rank 4 the RMSE is 0.952401795348
For rank 8 the RMSE is 0.959968407203
For rank 12 the RMSE is 0.953274504895
The best model was trained with rank 4


In [14]:
model = ALS.train(training_RDD, best_rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
predictions = model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print 'For testing data the RMSE is %s' % (error)

For testing data the RMSE is 0.943727800028


In [15]:
predictions.take(3)

[((80, 1084), 4.409276521842745),
 ((608, 1084), 3.865056484190859),
 ((514, 1084), 3.860466694422283)]

In [17]:
# Load the complete dataset file
complete_ratings_file = os.path.join(datasets_path, 'ml-latest', 'ratings.csv')
complete_ratings_raw_data = sc.textFile(complete_ratings_file)
complete_ratings_raw_data_header = complete_ratings_raw_data.take(1)[0]

# Parse
complete_ratings_data = complete_ratings_raw_data.filter(lambda line: line!=complete_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2]))).cache()
    
print "There are %s recommendations in the complete dataset" % (complete_ratings_data.count())

There are 24404096 recommendations in the complete dataset


In [18]:
training_RDD, test_RDD = complete_ratings_data.randomSplit([7, 3], seed=0L)

complete_model = ALS.train(training_RDD, best_rank, seed=seed, 
                           iterations=iterations, lambda_=regularization_parameter)

In [ ]:
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

predictions = complete_model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print 'For testing data the RMSE is %s' % (error)

In [15]:
complete_movies_file = os.path.join(datasets_path, 'ml-latest', 'movies.csv')
complete_movies_raw_data = sc.textFile(complete_movies_file)
complete_movies_raw_data_header = complete_movies_raw_data.take(1)[0]

# Parse
complete_movies_data = complete_movies_raw_data.filter(lambda line: line!=complete_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1],tokens[2])).cache()

complete_movies_titles = complete_movies_data.map(lambda x: (int(x[0]),x[1]))
    

In [55]:
movie_ID_with_ratings_RDD = (complete_ratings_data.map(lambda x: (x[1], x[2])).groupByKey())

In [41]:
fifty=complete_ratings_data.map(lambda x: (x[1], x[2])).take(5)

In [54]:
fiftyRdd=sc.parallelize(fifty)
fiftyRdd.take(5)

[(122, 2.0), (172, 1.0), (1221, 5.0), (1441, 4.0), (1609, 3.0)]

In [53]:
fiftyGroup=fiftyRdd.groupByKey()
fiftyGroup.collect()

[(122, <pyspark.resultiterable.ResultIterable at 0x10a30e910>),
 (172, <pyspark.resultiterable.ResultIterable at 0x10a30ef10>),
 (1441, <pyspark.resultiterable.ResultIterable at 0x10a30ef90>),
 (1221, <pyspark.resultiterable.ResultIterable at 0x10a30efd0>),
 (1609, <pyspark.resultiterable.ResultIterable at 0x10a31f050>)]

In [50]:

def get_counts_and_averages(ID_and_ratings_tuple):
    nratings = len(ID_and_ratings_tuple[1])
    print nratings
    return ID_and_ratings_tuple[0], (nratings, float(sum(x for x in ID_and_ratings_tuple[1])))

In [51]:
movie_ID_with_avg_ratings_RDD = fiftyGroup.map(get_counts_and_averages)

In [52]:
movie_ID_with_avg_ratings_RDD.take(5)

[(122, (1, 2.0)),
 (172, (1, 1.0)),
 (1441, (1, 4.0)),
 (1221, (1, 5.0)),
 (1609, (1, 3.0))]

In [21]:
movie_ID_with_avg_ratings_RDD.take(3)

[(122880, (10, 3.25)), (147460, (1, 2.5)), (131080, (53, 2.830188679245283))]

In [56]:
movie_ID_with_avg_ratings_RDD = movie_ID_with_ratings_RDD.map(get_counts_and_averages)
movie_rating_counts_RDD = movie_ID_with_avg_ratings_RDD.map(lambda x: (x[0], x[1][0]))

In [57]:
new_user_ID = 0

# The format of each line is (userID, movieID, rating)
new_user_ratings = [
     (0,260,9), # Star Wars (1977)
     (0,1,8), # Toy Story (1995)
     (0,16,7), # Casino (1995)
     (0,25,8), # Leaving Las Vegas (1995)
     (0,32,9), # Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
     (0,335,4), # Flintstones, The (1994)
     (0,379,3), # Timecop (1994)
     (0,296,7), # Pulp Fiction (1994)
     (0,858,10) , # Godfather, The (1972)
     (0,50,8) # Usual Suspects, The (1995)
    ]
new_user_ratings_RDD = sc.parallelize(new_user_ratings)
print 'New user ratings: %s' % new_user_ratings_RDD.take(10)

New user ratings: [(0, 260, 9), (0, 1, 8), (0, 16, 7), (0, 25, 8), (0, 32, 9), (0, 335, 4), (0, 379, 3), (0, 296, 7), (0, 858, 10), (0, 50, 8)]


In [58]:
complete_data_with_new_ratings_RDD = complete_ratings_data.union(new_user_ratings_RDD)

In [59]:
from time import time

t0 = time()
new_ratings_model = ALS.train(complete_data_with_new_ratings_RDD, best_rank, seed=seed, 
                              iterations=iterations, lambda_=regularization_parameter)
tt = time() - t0

print "New model trained in %s seconds" % round(tt,3)

New model trained in 127.59 seconds


In [60]:
new_user_ratings_ids = map(lambda x: x[1], new_user_ratings) # get just movie IDs
# keep just those not on the ID list (thanks Lei Li for spotting the error!)
new_user_unrated_movies_RDD = (complete_movies_data.filter(lambda x: x[0] not in new_user_ratings_ids).map(lambda x: (new_user_ID, x[0])))

# Use the input RDD, new_user_unrated_movies_RDD, with new_ratings_model.predictAll() to predict new ratings for the movies
new_user_recommendations_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)

In [70]:
new_user_recommendations_RDD.take(5)

[Rating(user=0, product=116688, rating=1.9665766591520009),
 Rating(user=0, product=7942, rating=6.502372110422872),
 Rating(user=0, product=149534, rating=6.655779417805983),
 Rating(user=0, product=103301, rating=1.5914932790213),
 Rating(user=0, product=121099, rating=2.3663611685532544)]

In [71]:
new_user_recommendations_rating_RDD.take(3)

[(116688, 1.9665766591520009),
 (7942, 6.502372110422872),
 (149534, 6.655779417805983)]

In [74]:
complete_movies_raw_data_header

u'movieId,title,genres'

In [73]:
movie_rating_counts_RDD.take(4)

[(122880, 10), (147460, 1), (131080, 53), (158380, 1)]

In [72]:
# Transform new_user_recommendations_RDD into pairs of the form (Movie ID, Predicted Rating)
new_user_recommendations_rating_RDD = new_user_recommendations_RDD.map(lambda x: (x.product, x.rating))
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_RDD.join(complete_movies_titles).join(movie_rating_counts_RDD)
new_user_recommendations_rating_title_and_count_RDD.take(3)

[(135168, ((6.001869266999439, u'Smithereens (1982)'), 5)),
 (65538, ((4.714814255016612, u'"Young Visiters'), 9)),
 (30723, ((5.743632967884787, u'Vincent & Theo (1990)'), 30))]

In [75]:
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_title_and_count_RDD.map(lambda r: (r[1][0][1], r[1][0][0], r[1][1]))


In [77]:
new_user_recommendations_rating_title_and_count_RDD.take(3)

[(u'Smithereens (1982)', 6.001869266999439, 5),
 (u'"Young Visiters', 4.714814255016612, 9),
 (u'Vincent & Theo (1990)', 5.743632967884787, 30)]

In [76]:
top_movies = new_user_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2]>=25).takeOrdered(25, key=lambda x: -x[1])

print ('TOP recommended movies (with more than 25 reviews):\n%s' %
        '\n'.join(map(str, top_movies)))

TOP recommended movies (with more than 25 reviews):
(u'Frozen Planet (2011)', 8.84152213107822, 239)
(u'Long Way Round (2004)', 8.815744812298474, 29)
(u'Mei and the Kittenbus (2002)', 8.69668631001355, 25)
(u'Death on the Staircase (Soup\xe7ons) (2004)', 8.646615078949761, 63)
(u'Planet Earth (2006)', 8.621617310110393, 193)
(u'"Civil War', 8.595081733299281, 380)
(u'"Godfather: Part II', 8.551911062921693, 34508)
(u'Dylan Moran Live: What It Is (2009)', 8.550363929529086, 59)
(u'The War (2007)', 8.51745825779463, 34)
(u'Band of Brothers (2001)', 8.512475690217443, 7833)
(u'Duck Amuck (1953)', 8.495506647023937, 120)
(u'"Lord of the Rings: The Fellowship of the Ring', 8.4592611641778, 51871)
(u'"Shawshank Redemption', 8.45652269283048, 84455)
(u'Cosmos (1980)', 8.45073889688232, 1740)
(u'"Lord of the Rings: The Return of the King', 8.42384785270097, 46446)
(u"Long Night's Journey Into Day (2000)", 8.403623290580663, 35)
(u'"I', 8.393612067566893, 45)
(u"Schindler's List (1993)", 8.382

In [81]:
my_movie = sc.parallelize([(0, 500)]) # Quiz Show (1994)
individual_movie_rating_RDD = new_ratings_model.predictAll(my_movie)
individual_movie_rating_RDD.take(1)

[Rating(user=0, product=500, rating=5.324391081735366)]

In [ ]:
from pyspark.mllib.recommendation import MatrixFactorizationModel

model_path = os.path.join('..', 'models', 'movie_lens_als')

# Save and load model
model.save(sc, model_path)
same_model = MatrixFactorizationModel.load(sc, model_path)